**Generate Questions using T5 tuned for Question generation**

**Model 1 :**  https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap

Citation:
@misc{mromero2021t5-base-finetuned-question-generation-ap,
  title={T5 (base) fine-tuned on SQUAD for QG via AP},
  author={Romero, Manuel},
  publisher={Hugging Face},
  journal={Hugging Face Hub},
  howpublished={\url{https://huggingface.co/mrm8488/t5-base-finetuned-question-generation-ap}},
  year={2021}
}

**FLAN-T5 is just given as reference but not used in the implementation. Future work.**


**Model 2:** https://huggingface.co/google/flan-t5-xxl

@misc{https://doi.org/10.48550/arxiv.2210.11416,
  doi = {10.48550/ARXIV.2210.11416},
  
  url = {https://arxiv.org/abs/2210.11416},
  
  author = {Chung, Hyung Won and Hou, Le and Longpre, Shayne and Zoph, Barret and Tay, Yi and Fedus, William and Li, Eric and Wang, Xuezhi and Dehghani, Mostafa and Brahma, Siddhartha and Webson, Albert and Gu, Shixiang Shane and Dai, Zhuyun and Suzgun, Mirac and Chen, Xinyun and Chowdhery, Aakanksha and Narang, Sharan and Mishra, Gaurav and Yu, Adams and Zhao, Vincent and Huang, Yanping and Dai, Andrew and Yu, Hongkun and Petrov, Slav and Chi, Ed H. and Dean, Jeff and Devlin, Jacob and Roberts, Adam and Zhou, Denny and Le, Quoc V. and Wei, Jason},
  
  keywords = {Machine Learning (cs.LG), Computation and Language (cs.CL), FOS: Computer and information sciences, FOS: Computer and information sciences},
  
  title = {Scaling Instruction-Finetuned Language Models},
  
  publisher = {arXiv},
  
  year = {2022},
  
  copyright = {Creative Commons Attribution 4.0 International}
}

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install --upgrade transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 9.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
# connect your personal google drive to store dataset and trained model
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

get_question(answer, context)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


'<pad> question: Who created the RuPERTa-base?</s>'

In [ ]:
def get_questions_for_source_t5(source):
  context_questions = []
  for index, row in source.iterrows():
    context = row['context'] 
    answer = row['keyword']
    question = get_question(answer, context)
    new_tuple = (context,answer,question)
    context_questions.append(new_tuple)
  return context_questions

In [ ]:
import pandas as pd
source=pd.read_excel("/content/gdrive/My Drive/CS 677 Project/dataset/context_keywords.xlsx", sheet_name='Sheet_name_1')

In [ ]:
source.head()

,Unnamed: 0,keyword,normalized_weight,context
0,0,form molecules,0.879163,Atoms are the building blocks that come togeth...
1,1,chemical bonding,0.841418,Atoms are the building blocks that come togeth...
2,2,building blocks,0.772690,Atoms are the building blocks that come togeth...
3,3,Atoms,0.763432,Atoms are the building blocks that come togeth...
4,4,Christian Guthier,0.743297,Atoms are the building blocks that come togeth...


In [ ]:
context_questions_tuple = get_questions_for_source_t5(source)
context_questions = pd.DataFrame(context_questions_tuple)
context_questions.to_excel("/content/gdrive/My Drive/CS 677 Project/dataset/context_questions.xlsx", sheet_name='Sheet_name_1')  

In [ ]:
# Example 1
context = "Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver rods indicate chemical bonds that hold the atoms together in a specific three-dimensional shape. (credit: modification of work by Christian Guthier)."
answer = "form molecules"
get_question(answer, context)


'<pad> question: Atoms are the building blocks that come together through chemical bonding to do what in the universe?</s>'

In [ ]:
# Example 2
context = "Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver rods indicate chemical bonds that hold the atoms together in a specific three-dimensional shape. (credit: modification of work by Christian Guthier)."
answer = "building blocks"
get_question(answer, context)

'<pad> question: What are atoms?</s>'

In [ ]:
# Example 3
context = "Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver rods indicate chemical bonds that hold the atoms together in a specific three-dimensional shape. (credit: modification of work by Christian Guthier)."
answer = "Atoms"
get_question(answer, context)

'<pad> question: What are the building blocks that come together through chemical bonding to form molecules in the universe?</s>'

In [ ]:
# Example 4
context = "Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver rods indicate chemical bonds that hold the atoms together in a specific three-dimensional shape. (credit: modification of work by Christian Guthier)."
answer = "molecules"
get_question(answer, context)

'<pad> question: Atoms are the building blocks that come together through chemical bonding to form what in the universe?</s>'

In [ ]:
#----------REFERENCE FOR FLAN-T5 -- NOT USED CURRENTLY -- FUTURE WORK ----------
# Generate Questions using flan-T5-xxl

#!pip install accelerate
#from transformers import T5Tokenizer, T5ForConditionalGeneration
#from accelerate import init_empty_weights
#
#tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xxl")
#model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xxl", device_map="auto", offload_folder="/content/gdrive/My Drive/CS677Working/offload")
#
#input_text = "Generate Question: Atoms are the building blocks that come together through chemical bonding to form molecules in the universe. In this model of a molecule, the atoms of carbon (black), hydrogen (white), nitrogen (blue), oxygen (red), and sulfur (yellow) are in proportional atomic size. The silver rods indicate chemical bonds that hold the atoms together in a specific three-dimensional shape. (credit: modification of work by Christian Guthier)."
#input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
#
#outputs = model.generate(input_ids)
#print(tokenizer.decode(outputs[0]))